In [ ]:
# Transformers installation
! pip install transformers datasets evaluate accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
traindata = pd.read_json('/content/drive/MyDrive/RisetQA/df_train.json')
valdata = pd.read_json('/content/drive/MyDrive/RisetQA/df_val.json')

In [ ]:
traindata[traindata['category']!='SPAN'].count()

,0
id,120
context,120
question,120
answer,61
category,120
span_start,120
span_end,120


In [ ]:
traindata = traindata[traindata['category'] == 'SPAN'].reset_index(drop=True)

In [ ]:
traindata = traindata.drop(columns=['span_start', 'span_end', 'category'])
traindata

,id,context,question,answer
0,629HsNl5Qf3THUq_CbmFrSx28,"Para pemuda pejuang, termasuk Chaerul Saleh, S...","Dengan siapa Chaerul Saleh, Sukarni, Wikana, d...",Ibrahim gelar Datuk Tan Malaka
1,2648F832Z8D-thJnrxy_ZoZ11111,Malaysia terdiri atas dua kawasan utama yang t...,Kapan angin muson timur laut berhembus?,Oktober hingga Februari
2,2561YDPNx6Nm5o6T0F-PGw-u2466,"Pada 16 Februari 1830, Diponegoro setuju untuk...",Siapa yang diutus oleh Jendral De Kock yang be...,Kolonel Jan Baptist Clereens
3,1431C0_ORK0ta8pDX6H-1VOp596,Kerajaan Buddhis Ligor segera kembali mengenda...,Apa yang chandrabanu lakukan setelah menguasai...,menggunakan tempat ini sebagai basis untuk men...
4,1234WIzl7ElikCCPk4gA-7ua3057,Sikap terhadap perbudakan juga berubah; semua ...,Kapan seluruh negara bagian Utara menghapuskan...,antara tahun 1780 dan 1804
...,...,...,...,...
3184,2158_3RZBuLXPIO5koP0W1QD4205,"Sistem parlemen dipuji, dibanding dengan siste...",Apa kekurangan dari sistem parlementer?,sering mengarah ke pemerintahan yang kurang st...
3185,1359ZuJdS_tCrQFa5IcePruQ795,Hatta berulang kali menolak Perjanjian Konsule...,Apa yang ingin diyakinkan Belanda kepada AS?,bahwa Indonesia adalah benteng komunis
3186,1559TG0OMQDF0IiIQ-iWWlLf1340,"Kemudian pada 9 Agustus 1945, Bung Hatta bersa...",Apakah tugas PPKI ?,melanjutkan hasil kerja BPUPKI dan menyiapkan ...
3187,2553rgsFqv862eNLs2TOpqys2483,Kuatnya pertahanan di Plered dibuktikan dengan...,Kapan terjadinya kegagalan pasukan Hindia Bela...,9 Juni 1826


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(traindata)

print("Train dataset:")
print(train_dataset)

Train dataset:
Dataset({
    features: ['id', 'context', 'question', 'answer'],
    num_rows: 3189
})


In [ ]:
from transformers import AutoTokenizer
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/xlm-roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
full_dataset = train_dataset.train_test_split(test_size=0.2)

In [ ]:
full_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answer'],
        num_rows: 2551
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answer'],
        num_rows: 638
    })
})

In [ ]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i in range(len(examples["answer"])):
        answer_text = examples["answer"][i]
        context = examples["context"][i]

        # Find the start and end character indices of the answer in the context
        start_char = context.find(answer_text)
        end_char = start_char + len(answer_text) if start_char != -1 else -1

        sequence_ids = inputs.sequence_ids(i)
        offset = offset_mapping[i]

        # Find the start and end token indices for the context
        context_start = next(k for k, v in enumerate(sequence_ids) if v == 1)
        context_end = len(sequence_ids) - 1 - next(k for k, v in enumerate(reversed(sequence_ids)) if v == 1)

        # If answer is not fully inside context or not found, label with (0, 0)
        if start_char == -1 or offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Token start index
            token_start_index = context_start
            while token_start_index <= context_end and offset[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            # Token end index
            token_end_index = context_end
            while token_end_index >= context_start and offset[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# Preprocess dataset
tokenized_dataset = full_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2551 [00:00<?, ? examples/s]

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answer', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 2551
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answer', 'input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 638
    })
})

In [ ]:
import pandas as pd
pd.DataFrame(tokenized_dataset["train"][:5])

,id,context,question,answer,input_ids,attention_mask,start_positions,end_positions
0,644VW80Ih4W_8lT57dCnHAu3021,Listyo pernah beberapa kali menduduki jabatan ...,Jabatan apa yang Listyo miliki setelah menjadi...,Wakapolrestabes Semarang,"[0, 59981, 2241, 119, 32036, 1410, 119288, 105...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",46,51
1,2483xn8QuuA6OqhzaYPTxh7I1820,India menjadi mitra wicara penuh ASEAN pada KT...,Dimanakah KTT ke-1 ASEAN-India dilaksanakan ?,"Phnom Penh, Kamboja","[0, 803, 4707, 6577, 341, 13739, 311, 5759, 13...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",58,65
2,25901E2zSJQfs-GE-OfTyHNi2415,Sekitar 2500 tahun SM Palestina merupakan nama...,Apa nama klasik wilayah Palestina?,Kan’an,"[0, 9795, 6831, 41292, 35588, 101403, 32, 2, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",44,46
3,3024oWqx7fEgnI85l7vJrQMv3808,"Di Asia, Amerika Serikat memimpin pendudukan J...",Siapakah yang memimpin pendudukan Jepang di Asia?,Amerika Serikat,"[0, 602, 180546, 119, 194406, 42651, 66, 63010...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",16,17
4,1616tJD99SBv5x6A67wD89lO1190,Pemilihan diadakan pada 30 Mei 1959 dengan Par...,Siapakah yang memenangkan Pemilu Singapura pad...,Partai Aksi Rakyat,"[0, 602, 180546, 119, 172646, 24199, 12175, 99...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",22,24


In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("FacebookAI/xlm-roberta-base")

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir="my_indo_2_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,2.067344
2,No log,1.476475
3,No log,1.381331


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=480, training_loss=2.25178705851237, metrics={'train_runtime': 655.3608, 'train_samples_per_second': 11.678, 'train_steps_per_second': 0.732, 'total_flos': 1499778059475456.0, 'train_loss': 2.25178705851237, 'epoch': 3.0})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...tent/my_indo_2_model/tokenizer.json:  98%|#########7| 16.7MB / 17.1MB            

  ...ndo_2_model/sentencepiece.bpe.model: 100%|##########| 5.07MB / 5.07MB            

  ...t/my_indo_2_model/model.safetensors:   0%|          | 1.25MB / 1.11GB            

  ...ents.1754011951.e47dc90d6bbc.2916.0:   8%|7         |   466B / 6.16kB            

  ...ents.1754012650.e47dc90d6bbc.2916.1:   8%|7         |  27.0B /   359B            

  ...t/my_indo_2_model/training_args.bin:   8%|7         |   401B / 5.30kB            

CommitInfo(commit_url='https://huggingface.co/mzuama/my_indo_2_model/commit/0a2f203314bbcff6f3c522b7de8c7d1e7b52effe', commit_message='End of training', commit_description='', oid='0a2f203314bbcff6f3c522b7de8c7d1e7b52effe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mzuama/my_indo_2_model', endpoint='https://huggingface.co', repo_type='model', repo_id='mzuama/my_indo_2_model'), pr_revision=None, pr_num=None)

In [ ]:
import evaluate

f1_metric = evaluate.load("f1")

# Make predictions on the test set
predictions = trainer.predict(tokenized_dataset["test"])

# Extract start and end position predictions
start_logits = predictions.predictions[0]
end_logits = predictions.predictions[1]

# Get the predicted start and end positions
predicted_start_positions = start_logits.argmax(axis=-1)
predicted_end_positions = end_logits.argmax(axis=-1)

# Get the true start and end positions directly from the dataset
true_start_positions = tokenized_dataset["test"]["start_positions"]
true_end_positions = tokenized_dataset["test"]["end_positions"]


# Calculate F1 score for span prediction
span_predictions = []
span_references = []

for i in range(len(predicted_start_positions)):
    span_predictions.append({
        "prediction_text": tokenizer.decode(tokenized_dataset["test"]["input_ids"][i][predicted_start_positions[i]:predicted_end_positions[i]+1]),
        "id": tokenized_dataset["test"]["id"][i]
    })
    span_references.append({
        "answers": {
            "answer_start": [tokenized_dataset["test"]["start_positions"][i]],
            "text": [tokenized_dataset["test"]["answer"][i]]
        },
        "id": tokenized_dataset["test"]["id"][i]
    })

# Note: The evaluate library's f1 metric for question answering expects a specific format.
# We need to adapt the predictions and references to match that format.
# A more robust evaluation would involve post-processing the predictions to get the actual answer text.
# For simplicity, we'll calculate a basic F1 based on token overlap for SPAN answers.

def calculate_span_f1(predictions, references):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for pred, ref in zip(predictions, references):
        pred_text = pred["prediction_text"]
        ref_text = ref["answers"]["text"][0]

        if pred_text == ref_text:
            true_positives += 1
        elif pred_text != "" and ref_text != "":
            false_positives += 1
            false_negatives += 1
        elif pred_text != "" and ref_text == "":
            false_positives += 1
        elif pred_text == "" and ref_text != "":
            false_negatives += 1

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return {"f1": f1, "precision": precision, "recall": recall}

span_f1_score = calculate_span_f1(span_predictions, span_references)
print(f"Span F1 Score: {span_f1_score}")

# For YESNO and UNANSWERABLE, we would need a different evaluation approach,
# potentially using accuracy or a custom metric based on the predicted answer type.
# This code focuses on the SPAN F1 score as requested.

Span F1 Score: {'f1': 0.43159609120521175, 'precision': 0.4491525423728814, 'recall': 0.4153605015673981}


In [ ]:
# Make predictions on the test set
predictions = trainer.predict(tokenized_dataset["test"])

# The predictions variable will contain the logits for start and end positions
# Access them like this:
start_logits = predictions.predictions[0]
end_logits = predictions.predictions[1]
# If your model also predicts answer types, they might be in predictions.predictions[2]

print("Predictions made successfully.")

Predictions made successfully.


In [ ]:
# Evaluate the model on the test set
eval_results = trainer.evaluate()
print(eval_results)

# Now you can proceed with calculating metrics like F1 score using the predictions
# from the previous prediction step (sJksA4tfJi0U)

{'eval_loss': 1.3813306093215942, 'eval_runtime': 13.1405, 'eval_samples_per_second': 48.552, 'eval_steps_per_second': 3.044, 'epoch': 3.0}


In [ ]:
def answer_question(question, context, model, tokenizer):
    """
    Answers a question based on the provided context using the trained model.
    """
    # Tokenize the input
    inputs = tokenizer(
        question,
        context,
        max_length=384,
        truncation="only_second",
        return_tensors="pt",
        padding="max_length",
    )

    # Move input tensors to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Get model predictions
    outputs = model(**inputs)

    # Get the predicted start and end positions
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely start and end positions
    answer_start = answer_start_scores.argmax()
    answer_end = answer_end_scores.argmax() + 1  # Add 1 to include the end token

    # Extract the answer span
    # Move input_ids back to CPU for decoding if necessary, although decode can often handle GPU tensors
    answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])


    # Handle cases where the answer is just special tokens
    if answer.startswith("[CLS]") or answer.endswith("[SEP]"):
        answer = "Unable to find answer in the text."

    return answer

# Example usage:
question = "apa itu presiden amerika serikat ?"
context = "Presiden Amerika Serikat (AS) merupakan salah satu posisi politik paling berpengaruh di dunia. Sebagai kepala negara dan pemerintahan, presiden AS memegang peran kunci dalam menentukan kebijakan domestik maupun hubungan internasional. Sejak kemerdekaan AS pada tahun 1776, negara ini telah dipimpin oleh berbagai presiden dengan visi dan kebijakan yang beragam, mulai dari George Washington hingga Joe Biden"
answer = answer_question(question, context, model, tokenizer)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

Question: apa itu presiden amerika serikat ?
Context: Presiden Amerika Serikat (AS) merupakan salah satu posisi politik paling berpengaruh di dunia. Sebagai kepala negara dan pemerintahan, presiden AS memegang peran kunci dalam menentukan kebijakan domestik maupun hubungan internasional. Sejak kemerdekaan AS pada tahun 1776, negara ini telah dipimpin oleh berbagai presiden dengan visi dan kebijakan yang beragam, mulai dari George Washington hingga Joe Biden
Answer: George Washington hingga Joe Biden


In [ ]:
# Example usage:
question = "Siapa presiden kedelapan RI?"
context = "Joko Widodo adlaah presiden ketuju RI. prabowo subianto adlaah presiden kedepalan RI"
answer = answer_question(question, context, model, tokenizer)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

Question: Siapa presiden kedelapan RI?
Context: Joko Widodo adlaah presiden ketuju RI. prabowo subianto adlaah presiden kedepalan RI
Answer: Joko Widodo


In [ ]:
import evaluate
squad_metric = evaluate.load("squad")

formatted_predictions = [
    {"id": ex["id"], "prediction_text": answer_question(ex["question"], ex["context"], model, tokenizer)}
    for ex in tokenized_dataset["test"]
]

references = [
    {"id": ex["id"], "answers": {"text": [ex["answer"]], "answer_start": [ex["start_positions"]]}}
    for ex in tokenized_dataset["test"]
]

results = squad_metric.compute(predictions=formatted_predictions, references=references)
print(results)


{'exact_match': 43.73040752351097, 'f1': 64.91700130794038}


In [ ]:
# Example usage:
question = "siapa detektif SMA?"
context = "Detektif Conan (名探偵コナン, Meitantei Konan), yang juga dikenal sebagai Case Closed dan Detective Conan, adalah seri manga detektif Jepang yang ditulis dan diilustrasikan oleh Gosho Aoyama. Serial ini diserialisasikan dalam majalah manga shōnen Weekly Shōnen Sunday yang diterbitkan oleh Shogakukan sejak Januari 1994, dengan bab-babnya dikumpulkan dalam beberapa volume tankōbon. Karena masalah hukum dengan nama Detective Conan, rilisan bahasa Inggris dari Funimation dan Viz diubah namanya menjadi Case Closed. Serial ini bercerita mengenai detektif SMA bernama Shinichi Kudo (atau Jimmy Kudo dalam beberapa terjemahan bahasa Inggris) yang tubuhnya menyusut menjadi kecil ketika menyelidiki sebuah organisasi misterius dan umumnya memecahkan banyak kasus dengan meniru suara dari ayah teman masa kecilnya dan berbagai karakter lain."
answer = answer_question(question, context, model, tokenizer)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

Question: siapa detektif SMA?
Context: Detektif Conan (名探偵コナン, Meitantei Konan), yang juga dikenal sebagai Case Closed dan Detective Conan, adalah seri manga detektif Jepang yang ditulis dan diilustrasikan oleh Gosho Aoyama. Serial ini diserialisasikan dalam majalah manga shōnen Weekly Shōnen Sunday yang diterbitkan oleh Shogakukan sejak Januari 1994, dengan bab-babnya dikumpulkan dalam beberapa volume tankōbon. Karena masalah hukum dengan nama Detective Conan, rilisan bahasa Inggris dari Funimation dan Viz diubah namanya menjadi Case Closed. Serial ini bercerita mengenai detektif SMA bernama Shinichi Kudo (atau Jimmy Kudo dalam beberapa terjemahan bahasa Inggris) yang tubuhnya menyusut menjadi kecil ketika menyelidiki sebuah organisasi misterius dan umumnya memecahkan banyak kasus dengan meniru suara dari ayah teman masa kecilnya dan berbagai karakter lain.
Answer: Shinichi Kudo


In [ ]:
# Example usage with a different question and context:
question = "Siapa detektif SMA?"
context = "Detektif Conan (名探偵コナン, Meitantei Konan), yang juga dikenal sebagai Case Closed dan Detective Conan, adalah seri manga detektif Jepang yang ditulis dan diilustrasikan oleh Gosho Aoyama. Serial ini diserialisasikan dalam majalah manga shōnen Weekly Shōnen Sunday yang diterbitkan oleh Shogakukan sejak Januari 1994, dengan bab-babnya dikumpulkan dalam beberapa volume tankōbon. Karena masalah hukum dengan nama Detective Conan, rilisan bahasa Inggris dari Funimation dan Viz diubah namanya menjadi Case Closed. Serial ini bercerita mengenai detektif SMA bernama Shinichi Kudo (atau Jimmy Kudo dalam beberapa terjemahan bahasa Inggris) yang tubuhnya menyusut menjadi kecil ketika menyelidiki sebuah organisasi misterius dan umumnya memecahkan banyak kasus dengan meniru suara dari ayah teman masa kecilnya dan berbagai karakter lain."
answer = answer_question(question, context, model, tokenizer)
print(f"Question: {question}")
print(f"Context: {context}")
print(f"Answer: {answer}")

Question: Siapa detektif SMA?
Context: Detektif Conan (名探偵コナン, Meitantei Konan), yang juga dikenal sebagai Case Closed dan Detective Conan, adalah seri manga detektif Jepang yang ditulis dan diilustrasikan oleh Gosho Aoyama. Serial ini diserialisasikan dalam majalah manga shōnen Weekly Shōnen Sunday yang diterbitkan oleh Shogakukan sejak Januari 1994, dengan bab-babnya dikumpulkan dalam beberapa volume tankōbon. Karena masalah hukum dengan nama Detective Conan, rilisan bahasa Inggris dari Funimation dan Viz diubah namanya menjadi Case Closed. Serial ini bercerita mengenai detektif SMA bernama Shinichi Kudo (atau Jimmy Kudo dalam beberapa terjemahan bahasa Inggris) yang tubuhnya menyusut menjadi kecil ketika menyelidiki sebuah organisasi misterius dan umumnya memecahkan banyak kasus dengan meniru suara dari ayah teman masa kecilnya dan berbagai karakter lain.
Answer: Shinichi Kudo


In [ ]:
context="Bacharuddin Jusuf Habibie[1] (25 Juni 1936 – 11 September 2019)[2][a] adalah Presiden Indonesia ketiga yang menjabat sejak tahun 1998 sampai 1999, menggantikan Soeharto yang mengundurkan diri dari jabatan presiden pada tanggal 21 Mei 1998.[3][4] Sebelumnya, ia menjabat sebagai Wakil Presiden Indonesia ketujuh. Sebelum memasuki dunia politik, Habibie dikenal luas sebagai seorang profesor dan ilmuwan dalam teknologi aviasi internasional dan satu-satunya presiden Indonesia hingga saat ini yang berlatarbelakang teknokrat.B.J. Habibie kemudian digantikan oleh Abdurrahman Wahid[5] (Gus Dur) yang terpilih sebagai presiden pada 20 Oktober 1999 oleh MPR hasil Pemilu 1999. Dengan menjabat selama 2 bulan dan 7 hari (sebagai wakil presiden) dan juga selama 1 tahun dan 5 bulan (sebagai presiden), B. J. Habibie merupakan Wakil Presiden dan juga Presiden Indonesia dengan masa jabatan terpendek.[6] B. J. Habibie merupakan presiden Indonesia pertama yang terlahir di luar Jawa, tepatnya di Parepare, Sulawesi Selatan. Ia berasal dari etnis Bugis[7][8][9][10]-Gorontalo[11][12] dari garis keturunan ayahnya yang berasal dari Kabila, Gorontalo dan etnis Jawa dari ibunya yang berasal dari Yogyakarta.["
questions_to_ask = [
    "Siapa Habibie?",
    "Siapa soeharto?",
    "Siapa wakil presiden ketuju?",
    "siapa pengganti Habibie?"
]

for question in questions_to_ask:
    answer = answer_question(question, context, model, tokenizer)
    print(f"Question: {question}")
    print(f"Context: {context}")
    print(f"Answer: {answer}")
    print("-" * 20) # Separator for clarity

Question: Siapa Habibie?
Context: Bacharuddin Jusuf Habibie[1] (25 Juni 1936 – 11 September 2019)[2][a] adalah Presiden Indonesia ketiga yang menjabat sejak tahun 1998 sampai 1999, menggantikan Soeharto yang mengundurkan diri dari jabatan presiden pada tanggal 21 Mei 1998.[3][4] Sebelumnya, ia menjabat sebagai Wakil Presiden Indonesia ketujuh. Sebelum memasuki dunia politik, Habibie dikenal luas sebagai seorang profesor dan ilmuwan dalam teknologi aviasi internasional dan satu-satunya presiden Indonesia hingga saat ini yang berlatarbelakang teknokrat.B.J. Habibie kemudian digantikan oleh Abdurrahman Wahid[5] (Gus Dur) yang terpilih sebagai presiden pada 20 Oktober 1999 oleh MPR hasil Pemilu 1999. Dengan menjabat selama 2 bulan dan 7 hari (sebagai wakil presiden) dan juga selama 1 tahun dan 5 bulan (sebagai presiden), B. J. Habibie merupakan Wakil Presiden dan juga Presiden Indonesia dengan masa jabatan terpendek.[6] B. J. Habibie merupakan presiden Indonesia pertama yang terlahir di lu

In [ ]:
context = """
Bacharuddin Jusuf Habibie (25 Juni 1936 – 11 September 2019) adalah Presiden ketiga Republik Indonesia yang menjabat dari tahun 1998 hingga 1999. Ia menggantikan Presiden Soeharto yang mengundurkan diri pada 21 Mei 1998. Sebelum menjadi presiden, Habibie menjabat sebagai Wakil Presiden Indonesia ketujuh selama 2 bulan dan 7 hari, menjadikannya wakil presiden dengan masa jabatan terpendek.

Habibie dikenal luas sebagai seorang profesor dan ilmuwan di bidang teknologi penerbangan internasional, dan merupakan satu-satunya presiden Indonesia yang berlatar belakang teknokrat. Setelah masa kepresidenannya berakhir, ia digantikan oleh Abdurrahman Wahid (Gus Dur) yang terpilih oleh MPR hasil Pemilu 1999.

Habibie merupakan Presiden Indonesia pertama yang lahir di luar Pulau Jawa, tepatnya di Parepare, Sulawesi Selatan. Ia berasal dari keluarga campuran etnis Bugis-Gorontalo dari pihak ayahnya yang berasal dari Kabila, Gorontalo, dan etnis Jawa dari pihak ibunya yang berasal dari Yogyakarta.

Nama istri Bacharuddin Jusuf Habibie adalah Hasri Ainun Besari.
"""
questions_to_ask = [
    "Siapa Bacharuddin Jusuf Habibie?",
    "Kapan BJ Habibie menjabat sebagai Presiden Indonesia?",
    "Mengapa BJ Habibie menjadi Presiden Indonesia?",
    "Berapa lama BJ Habibie menjabat sebagai Wakil Presiden?",
    "Apa latar belakang profesi BJ Habibie sebelum menjadi Presiden?",
    "Siapa yang menggantikan BJ Habibie sebagai Presiden Indonesia?",
    "Apa yang membedakan BJ Habibie dari presiden Indonesia lainnya?",
    "Dari mana asal daerah kelahiran BJ Habibie?",
    "Apa etnis dari BJ Habibie?",
    "Siapa nama istri BJ Habibie?"
]


for question in questions_to_ask:
    answer = answer_question(question, context, model, tokenizer)
    print(f"Question: {question}")
    print()
    print(f"Answer: {answer}")
    print()
    print("-" * 20)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Question: Siapa Bacharuddin Jusuf Habibie?

Answer: Presiden ketiga Republik Indonesia yang menjabat dari tahun 1998 hingga 1999. Ia menggantikan Presiden Soeharto yang mengundurkan diri pada 21 Mei 1998. Sebelum menjadi presiden, Habibie menjabat sebagai Wakil Presiden Indonesia ketujuh selama 2 bulan dan 7 hari, menjadikannya wakil presiden dengan masa jabatan terpendek. Habibie dikenal luas sebagai seorang profesor dan ilmuwan di bidang teknologi penerbangan internasional, dan merupakan satu-satunya presiden Indonesia yang berlatar belakang teknokrat. Setelah masa kepresidenannya berakhir, ia digantikan oleh Abdurrahman Wahid (Gus Dur) yang terpilih oleh MPR hasil Pemilu 1999. Habibie merupakan Presiden Indonesia pertama yang lahir di luar Pulau Jawa, tepatnya di Parepare, Sulawesi Selatan. Ia berasal dari keluarga campuran etnis Bugis-Gorontalo dari pihak ayahnya yang berasal dari Kabila, Gorontalo, dan etnis Jawa dari pihak ibunya yang berasal dari Yogyakarta. Nama istri Bacharudd

In [ ]:
context = """
Bacharuddin Jusuf Habibie (25 Juni 1936 – 11 September 2019) adalah Presiden ketiga Republik Indonesia yang menjabat dari tahun 1998 hingga 1999. Ia menggantikan Presiden Soeharto yang mengundurkan diri pada 21 Mei 1998 setelah terjadi krisis politik dan ekonomi yang memuncak dalam Reformasi 1998.

Sebelum menjadi presiden, Habibie menjabat sebagai Wakil Presiden Indonesia selama 2 bulan dan 7 hari, menjadikannya sebagai wakil presiden dengan masa jabatan terpendek. Sebelumnya, ia menjabat sebagai Menteri Riset dan Teknologi Indonesia selama lebih dari 20 tahun.

Habibie dikenal luas sebagai seorang profesor dan ilmuwan di bidang teknologi penerbangan. Ia pernah bekerja di perusahaan penerbangan Jerman, Messerschmitt-Bölkow-Blohm (MBB), di mana ia berkontribusi dalam pengembangan pesawat. Ia juga dikenal sebagai pencetus teori crack propagation dalam bidang teknik material.

Habibie merupakan satu-satunya Presiden Indonesia yang berlatar belakang teknokrat dan bukan berasal dari militer atau partai politik utama. Ia lahir di Parepare, Sulawesi Selatan, dan merupakan Presiden pertama yang lahir di luar Pulau Jawa. Ayahnya berasal dari Kabila, Gorontalo, dan ibunya dari Yogyakarta, Jawa Tengah.

Habibie adalah tokoh penting dalam sejarah modern Indonesia. Masa pemerintahannya ditandai dengan upaya demokratisasi, seperti pembebasan tahanan politik, liberalisasi pers, dan pelaksanaan pemilu yang lebih demokratis pada tahun 1999. Ia juga memberikan kesempatan bagi Timor Timur untuk mengadakan referendum, yang kemudian berujung pada kemerdekaan negara tersebut.

Setelah masa kepresidenannya, Habibie tetap aktif dalam kegiatan intelektual dan sosial. Ia mendirikan Habibie Center, sebuah lembaga pemikiran (think tank) yang fokus pada demokrasi dan hak asasi manusia. Ia meninggal dunia pada 11 September 2019 di Jakarta karena komplikasi jantung.

Nama istri Bacharuddin Jusuf Habibie adalah Hasri Ainun Besari. Kisah cinta mereka diabadikan dalam film berjudul "Habibie & Ainun" yang dirilis tahun 2012 dan sangat populer di kalangan masyarakat Indonesia.
"""
questions_to_ask = [
    "Siapa Bacharuddin Jusuf Habibie?",
    "Kapan BJ Habibie menjabat sebagai Presiden Indonesia?",
    "Mengapa BJ Habibie menjadi Presiden Indonesia?",
    "Berapa lama BJ Habibie menjabat sebagai Wakil Presiden?",
    "Apa latar belakang profesi BJ Habibie sebelum menjadi Presiden?",
    "Apa kontribusi Habibie di bidang teknologi penerbangan?",
    "Siapa yang menggantikan BJ Habibie sebagai Presiden Indonesia?",
    "Apa yang membedakan BJ Habibie dari presiden Indonesia lainnya?",
    "Dari mana asal daerah kelahiran BJ Habibie?",
    "Apa etnis dari BJ Habibie?",
    "Siapa nama istri BJ Habibie?",
    "Apa nama lembaga yang didirikan oleh BJ Habibie setelah menjabat sebagai Presiden?",
    "Apa peran BJ Habibie dalam demokratisasi Indonesia?",
    "Apa kebijakan penting BJ Habibie terkait Timor Timur?",
    "Dalam bidang apa teori crack propagation ditemukan oleh Habibie?",
    "Di mana BJ Habibie bekerja sebelum kembali ke Indonesia?",
    "Kapan BJ Habibie meninggal dan apa penyebabnya?",
    "Apa nama film yang menceritakan kisah cinta BJ Habibie dan istrinya?",
    "Apa kontribusi Habibie terhadap pemilu 1999?",
    "Apa latar belakang pendidikan BJ Habibie?"
]

for question in questions_to_ask:
    answer = answer_question(question, context, model, tokenizer)
    print(f"Question: {question}")
    print()
    print(f"Answer: {answer}")
    print()
    print("-" * 20)

Question: Siapa Bacharuddin Jusuf Habibie?

Answer: Hasri Ainun Besari

--------------------
Question: Kapan BJ Habibie menjabat sebagai Presiden Indonesia?

Answer: 2 bulan dan 7 hari

--------------------
Question: Mengapa BJ Habibie menjadi Presiden Indonesia?

Answer: 2 bulan dan 7 hari, menjadikannya sebagai wakil presiden dengan masa jabatan terpendek. Sebelumnya, ia menjabat sebagai Menteri Riset dan Teknologi Indonesia selama lebih dari 20 tahun. Habibie dikenal luas sebagai seorang profesor dan ilmuwan di bidang teknologi penerbangan. Ia pernah bekerja di perusahaan penerbangan Jerman, Messerschmitt-Bölkow-Blohm (MBB), di mana ia berkontribusi dalam pengembangan pesawat. Ia juga dikenal sebagai pencetus teori crack propagation dalam bidang teknik material. Habibie merupakan satu-satunya Presiden Indonesia yang berlatar belakang teknokrat dan bukan berasal dari militer atau partai politik utama. Ia lahir di Parepare, Sulawesi Selatan, dan merupakan Presiden pertama yang lahir d

In [ ]:
context = """
Kecerdasan buatan atau Artificial Intelligence (AI) adalah bidang ilmu komputer yang bertujuan untuk menciptakan sistem yang mampu melakukan tugas-tugas yang biasanya memerlukan kecerdasan manusia, seperti pengenalan suara, penglihatan komputer, pemahaman bahasa alami, dan pengambilan keputusan.

Istilah "Artificial Intelligence" pertama kali diperkenalkan pada tahun 1956 oleh John McCarthy dalam konferensi Dartmouth Summer Research Project on Artificial Intelligence. Konferensi ini dianggap sebagai titik awal resmi dari bidang AI.

Pada dekade 1950–1970, riset AI berkembang pesat karena optimisme bahwa komputer dapat segera meniru kecerdasan manusia. Namun, pada 1970-an, AI mengalami kemunduran yang dikenal sebagai "AI winter" karena keterbatasan komputasi dan kurangnya hasil nyata.

AI mulai bangkit kembali pada 1980-an dengan munculnya sistem pakar, dan pada 2010-an, AI kembali mengalami lonjakan berkat kemajuan dalam deep learning, ketersediaan data besar (big data), dan peningkatan kemampuan komputasi GPU.

Saat ini, AI diterapkan secara luas dalam berbagai bidang, seperti kesehatan, keuangan, transportasi, dan hiburan. Model-model seperti ChatGPT, Google DeepMind, dan AlphaGo adalah contoh implementasi AI canggih masa kini.
"""

questions_to_ask = [
    "Apa itu kecerdasan buatan atau AI?",
    "Siapa yang pertama kali memperkenalkan istilah Artificial Intelligence?",
    "Kapan istilah AI pertama kali diperkenalkan?",
    "Apa yang terjadi pada AI pada tahun 1970-an?",
    "Apa penyebab kebangkitan kembali AI di tahun 2010-an?",
    "Apa contoh aplikasi AI di masa kini?",
    "Apa itu AI winter?",
    "Apa kontribusi John McCarthy dalam sejarah AI?",
    "Apa peran deep learning dalam perkembangan AI modern?",
    "Dalam bidang apa saja AI diterapkan saat ini?"
]

for question in questions_to_ask:
    answer = answer_question(question, context, model, tokenizer)
    print(f"Question: {question}")
    print()
    print(f"Answer: {answer}")
    print()
    print("-" * 20)


Question: Apa itu kecerdasan buatan atau AI?

Answer: bidang ilmu komputer yang bertujuan untuk menciptakan sistem yang mampu melakukan tugas-tugas yang biasanya memerlukan kecerdasan manusia, seperti pengenalan suara, penglihatan komputer, pemahaman bahasa alami, dan pengambilan keputusan

--------------------
Question: Siapa yang pertama kali memperkenalkan istilah Artificial Intelligence?

Answer: John McCarthy

--------------------
Question: Kapan istilah AI pertama kali diperkenalkan?

Answer: tahun 1956

--------------------
Question: Apa yang terjadi pada AI pada tahun 1970-an?

Answer: keterbatasan komputasi dan kurangnya hasil nyata. AI mulai bangkit kembali pada 1980-an dengan munculnya sistem pakar

--------------------
Question: Apa penyebab kebangkitan kembali AI di tahun 2010-an?

Answer: 

--------------------
Question: Apa contoh aplikasi AI di masa kini?

Answer: Model-model seperti ChatGPT, Google DeepMind, dan AlphaGo

--------------------
Question: Apa itu AI winter

In [ ]:
context = """
Republik Rakyat Tiongkok, dikenal sebagai China, adalah negara berdaulat yang terletak di Asia Timur dan merupakan negara terpadat di dunia dengan populasi lebih dari 1,4 miliar jiwa. Ibu kota China adalah Beijing, sementara kota terbesar dan pusat keuangan adalah Shanghai. China memiliki sejarah peradaban yang sangat panjang, dimulai dari dinasti pertama, yaitu Dinasti Xia, sekitar 2100 SM.

Pada tahun 1949, Partai Komunis China yang dipimpin Mao Zedong mendirikan Republik Rakyat Tiongkok setelah mengalahkan Partai Nasionalis dalam Perang Saudara China. Sejak saat itu, China mengalami berbagai reformasi sosial, politik, dan ekonomi yang signifikan, terutama setelah kebijakan reformasi dan keterbukaan ekonomi yang dimulai oleh Deng Xiaoping pada akhir 1970-an.

China merupakan salah satu kekuatan ekonomi terbesar di dunia dan dikenal karena kemajuan pesatnya dalam bidang teknologi, infrastruktur, dan manufaktur. China juga memiliki kekuatan militer terbesar di dunia dan menjadi anggota tetap Dewan Keamanan PBB.

Bahasa resmi China adalah Mandarin, dan mata uang resminya adalah Yuan Renminbi (RMB). Negara ini memiliki sistem politik satu partai di bawah kekuasaan Partai Komunis China.
"""


In [ ]:
questions_to_ask = [
    "Apa nama resmi dari negara China?",
    "Di mana letak geografis negara China?",
    "Berapa jumlah penduduk China?",
    "Apa ibu kota negara China?",
    "Siapa yang mendirikan Republik Rakyat Tiongkok?",
    "Kapan Republik Rakyat Tiongkok didirikan?",
    "Apa yang terjadi setelah tahun 1970-an di China?",
    "Apa bahasa resmi yang digunakan di China?",
    "Apa mata uang resmi China?",
    "Partai politik apa yang berkuasa di China saat ini?",
    "Apa kota terbesar dan pusat keuangan di China?",
    "Bagaimana posisi militer China di dunia?",
    "Siapa tokoh penting di balik reformasi ekonomi China?",
    "China adalah anggota tetap dari lembaga internasional apa?",
    "Apa sistem pemerintahan yang dianut China?"
]

for question in questions_to_ask:
    answer = answer_question(question, context, model, tokenizer)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-" * 40)


Question: Apa nama resmi dari negara China?
Answer: Mandarin
----------------------------------------
Question: Di mana letak geografis negara China?
Answer: Asia Timur
----------------------------------------
Question: Berapa jumlah penduduk China?
Answer: lebih dari 1,4 miliar jiwa
----------------------------------------
Question: Apa ibu kota negara China?
Answer: Beijing
----------------------------------------
Question: Siapa yang mendirikan Republik Rakyat Tiongkok?
Answer: Partai Komunis China yang dipimpin Mao Zedong
----------------------------------------
Question: Kapan Republik Rakyat Tiongkok didirikan?
Answer: tahun 1949
----------------------------------------
Question: Apa yang terjadi setelah tahun 1970-an di China?
Answer: kebijakan reformasi dan keterbukaan ekonomi yang dimulai oleh Deng Xiaoping
----------------------------------------
Question: Apa bahasa resmi yang digunakan di China?
Answer: Mandarin
----------------------------------------
Question: Apa mata ua

In [ ]:
trainer.save_model("/content/drive/MyDrive/my_indo_QA2_model")

load model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Path where you saved the model
model_path = "/content/drive/MyDrive/my_trained_qa_model"

# Load the model and tokenizer
loaded_model = AutoModelForQuestionAnswering.from_pretrained(model_path)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)

print("Model and tokenizer loaded successfully.")